In [1]:
# ==== Install Dependencies

!pip install -q efficientnet_pytorch
!pip install -q albumentations
!pip install -q pytorch-fanatics 
!pip install -q pytorch_ranger

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# ==== Import Libraries

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
import seaborn as sns
import random
import os


import albumentations as aug
from albumentations.pytorch.transforms import ToTensor
import matplotlib.pyplot as plt

from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset,DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

# from pytorch_fanatics.dataloader import Cloader
from pytorch_fanatics.utils import EarlyStop 
from pytorch_fanatics.trainer import Trainer
from pytorch_fanatics.logger import Logger

import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from pytorch_ranger import Ranger

In [3]:
from glob import glob

In [4]:
train_path = []
train_label = []

val_path = []
val_label = []

In [5]:
for i in glob("../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/MildDemented/*"):
    train_path.append(i)
    train_label.append(0)
    
for i in glob("../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/ModerateDemented/*"):
    train_path.append(i)
    train_label.append(1)

for i in glob("../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/NonDemented/*"):
    train_path.append(i)
    train_label.append(2)    
    
for i in glob("../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/VeryMildDemented/*"):
    train_path.append(i)
    train_label.append(3)

for i in glob("../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/MildDemented/*"):
    val_path.append(i)
    val_label.append(0)
    
for i in glob("../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/ModerateDemented/*"):
    val_path.append(i)
    val_label.append(1)

for i in glob("../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/NonDemented/*"):
    val_path.append(i)
    val_label.append(2)    
    
for i in glob("../input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/VeryMildDemented/*"):
    val_path.append(i)
    val_label.append(3)    
    

In [6]:
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [7]:
# ===== Augmentations

mean       = (0.485, 0.456, 0.406)
std        = (0.229, 0.224, 0.225)
train_tfms = aug.Compose([
            aug.Resize(224,224),
            aug.Flip(p=0.5),
            aug.ShiftScaleRotate(rotate_limit=(-45,45)),
            aug.GaussNoise(p=0.35),
            aug.Normalize(mean,std,max_pixel_value=255.0,always_apply=True),
            ])

test_tfms  = aug.Compose([
            aug.Resize(224,224),
            aug.Normalize(mean,std,max_pixel_value=255.0,always_apply=True),
            ])

In [8]:
import numpy as np
from PIL import Image
from PIL import ImageFile   #ImageFile contains support for PIL to open and save images
ImageFile.LOAD_TRUNCATED_IMAGES=True  #If image is truncated(or corrupt) then also load it..
import torch

class Cloader:
    def __init__(self,image_path,targets,resize=None,transforms=None):
        self.image_path=image_path
        self.targets=targets
        self.resize=resize
        self.transforms=transforms
        
    def __len__(self):
        return len(self.image_path)
    
    def __getitem__(self,idx):
        image = Image.open(self.image_path[idx]).convert('RGB')
        targets = self.targets[idx]
        if self.resize is not None:
            image = image.resize(
                (self.resize[1], self.resize[0]), resample=Image.BILINEAR
            )
        image = np.array(image)
        if self.transforms is not None:
            augmented = self.transforms(image=image)
            image = augmented["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.long),
        }


In [9]:

train_dataset    = Cloader(train_path,train_label,None,train_tfms)
test_dataset     = Cloader(val_path,val_label,None,test_tfms)

train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=4)
val_dataloader   = DataLoader(test_dataset,batch_size=32,shuffle=False,num_workers=4)

device           = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [10]:
# ===== Define model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.base_model = EfficientNet.from_pretrained('efficientnet-b0',num_classes=4)
    def forward(self, image, targets):
        batch_size, _, _, _ = image.shape
        out = self.base_model(image)
        targets = torch.tensor(targets,dtype=torch.int64)
        loss = nn.CrossEntropyLoss()(out.view(batch_size,4), targets)
        return out, loss

model = Net()
model.to(device);

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


In [11]:
def softmax(array):
    return np.exp(array)/np.sum(np.exp(array),axis=1).reshape(-1,1)

In [12]:
optimizer = Ranger(model.parameters(),lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer,factor=0.8, mode="min",min_lr= 5e-6, patience=2)

trainer   = Trainer(model=model,optimizer=optimizer,device=device,val_scheduler=scheduler)
logger    = Logger()

es        = EarlyStop(patience=5,mode="min") # mode = min to minimise loss

In [13]:
epochs = 50
best_loss = np.Inf

for epoch in range(epochs):
    logger.write(f"+ ===== Epoch {epoch+1}/{epochs} ===== +")
    train_loss              = trainer.train(train_dataloader)
    y_true,y_pred ,val_loss = trainer.evaluate(val_dataloader)
    y_pred                  = softmax(y_pred)
    accuracy                = accuracy_score(y_true,np.argmax(y_pred,axis=1))
    logger.write(f"train_loss {train_loss} val_loss {val_loss} ")
    logger.write(f"val accuracy_score {accuracy} ")
    logger.write(" ")
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(),'best.pth')


+ ===== Epoch 1/50 ===== +



train_loss 1.332680420845932 val_loss 1.1896885231137277 
val accuracy_score 0.5027365129007036 
 
+ ===== Epoch 2/50 ===== +



train_loss 1.1195236500005545 val_loss 1.0179789379239081 
val accuracy_score 0.5230648944487881 
 
+ ===== Epoch 3/50 ===== +



train_loss 0.9707580941804449 val_loss 0.9744964979588983 
val accuracy_score 0.5418295543393276 
 
+ ===== Epoch 4/50 ===== +



train_loss 0.8957206754951004 val_loss 0.9223350264132025 
val accuracy_score 0.5598123534010946 
 
+ ===== Epoch 5/50 ===== +


...


train_loss 0.1449260465014055 val_loss 0.8009914431720974 
val accuracy_score 0.7756059421422987 
 
+ ===== Epoch 47/50 ===== +



train_loss 0.1556706982115226 val_loss 0.799178710207343 
val accuracy_score 0.7740422204847537 
 
+ ===== Epoch 48/50 ===== +



train_loss 0.1466179146020678 val_loss 0.8009882006794213 
val accuracy_score 0.7763878029710711 
 
+ ===== Epoch 49/50 ===== +



train_loss 0.1559488419790446 val_loss 0.8110296070575714 
val accuracy_score 0.7709147771696638 
 
+ ===== Epoch 50/50 ===== +



train_loss 0.14609237668908528 val_loss 0.7981901068240403 
val accuracy_score 0.7740422204847537 
 
